In [ ]:
from graphframes import GraphFrame
from pyspark.sql.functions import sum
from graphframes.lib import AggregateMessages
from pyspark.sql.types import IntegerType

In [ ]:
inputPathVertexes = "/data/students/bigdata-01QYD/ex_data/Ex57b/data/vertexes.csv"
inputPathEdges = "/data/students/bigdata-01QYD/ex_data/Ex57b/data/edges.csv"
outputPath = "resOut_ex57b/"

In [ ]:
# Read the content of vertexes.csv
vDF = spark.read.load(inputPathVertexes,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
#vDF.printSchema()
#vDF.show()

In [ ]:
# Read the content of edges.csv
eDF = spark.read.load(inputPathEdges,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
#eDF.printSchema()
#eDF.show()

In [ ]:
# Add to each node/vertex a new "feature": AgeLess35
# AgeLess35
def funcAgeLess35(age):
    if (age<35):
        return 1;
    else:
        return 0;
    
spark.udf.register("funcAgeLess35", funcAgeLess35, IntegerType())

vDFnew = vDF.selectExpr("*", "funcAgeLess35(age) as AgeLess35")

In [ ]:
vDFnew.printSchema()
vDFnew.show()

In [ ]:
# Create the input graph
g = GraphFrame(vDFnew, eDF)

In [ ]:
# For each user, sum the values of AgeLess35 of neighbors

# Send AgeLess35 of each source of an edge to its destination
msgToDst = AggregateMessages.src["AgeLess35"]

In [ ]:
# Aggregate messages 
aggAgeLess35 = g.aggregateMessages(sum(AggregateMessages.msg),\
                             sendToSrc=None,\
                             sendToDst=msgToDst).\
withColumnRenamed("sum(MSG)", "numNeighborsLess35")

In [ ]:
aggAgeLess35.show()

In [ ]:
selecteUsersDF = aggAgeLess35\
.filter("numNeighborsLess35>0")

In [ ]:
selecteUsersDF.show()

In [ ]:
# Save the result in the output folder
selecteUsersDF.write.csv(outputPath, header=True)